In [1]:
pip install matplotlib

  Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 178.5 kB/s eta 0:00:000:00:02m eta 0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 196.5 kB/s eta 0:00:001m193.9 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 183.4 kB/s eta 0:00:001m174.5 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 1.3 MB/s eta 0:00:000:00:010:00:01:01m
Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ollama

  Using cached pydantic-2.10.5-py3-none-any.whl.metadata (30 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached pydantic-2.10.5-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
dataset = []
with open('Weekly Menu from 27-11-2023 to 03-12-2023.csv','r') as file:
    dataset = file.readlines()
    print(f"loaded {len(dataset)} entries")

loaded 82 entries


In [11]:
import ollama
EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

VECTOR_DB = []

def add_chunk_to_database(chunk):
    embedding = ollama.embed(model=EMBEDDING_MODEL,input=chunk)['embeddings'][0]
    VECTOR_DB.append((chunk,embedding))


In [22]:
chunk_size = 13
for i in range(0, len(dataset), chunk_size):
    chunk = dataset[i:i+chunk_size]  # Group 13 lines together
    add_chunk_to_database(chunk)
    print(f'Added chunk {i//chunk_size + 1}/{-(-len(dataset) // chunk_size)} to the database')


Added chunk 1/7 to the database
Added chunk 2/7 to the database
Added chunk 3/7 to the database
Added chunk 4/7 to the database
Added chunk 5/7 to the database
Added chunk 6/7 to the database
Added chunk 7/7 to the database


In [23]:
def cosine_similarity(a,b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)
    

In [24]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]


In [25]:
#Generation Phase
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
'''

Ask me a question:  Whats for lunch on wednesday


Retrieved knowledge:
 - (similarity: 0.78) Tuesday,VELLIYAPAM,POORI,VEG MANCHOW SOUP,KOSLOW SANDWICH

 - (similarity: 0.78) Tuesday,VELLIYAPAM,POORI,VEG MANCHOW SOUP,KOSLOW SANDWICH

 - (similarity: 0.70) Wednesday,MYSORE BONDA,ONIONS,PULKA,SAMOSA



In [26]:
stream = ollama.chat(
    model = LANGUAGE_MODEL,
    messages =[
        {'role':'system','content':instruction_prompt},
        {'role':'user','content':input_query},
    ],
    stream = True,
)

print('Chatbot response:')
for chunk in stream:
    print(chunk['message']['content'],end='',flush=True)

Chatbot response:
Since it's Wednesday and the context is:

Wednesday,MYSORE BONDA,ONIONS,PULKA,SAMOSA

We can conclude that there will be lunch consisting of Mysore Bonda, onions, pulka ( possibly referring to a specific type of fried rice or snack), and Samosa.